# 0. Импорт библиотек, задание функций

In [85]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split

# 1. Загрузка датасета, разбиение на train test, создание матриц

In [10]:
ratings = pd.read_csv('movie_data/ratings.csv')
movies = pd.read_csv('movie_data/movies.csv')

print('Размер датасета:', ratings.shape)
print('Уникальные пользователи:', ratings['userId'].nunique())
print('Уникальные фильмы:', ratings['movieId'].nunique())

Размер датасета: (100836, 4)
Уникальные пользователи: 610
Уникальные фильмы: 9724


In [12]:
ratings_sorted = ratings.sort_values(by='timestamp')
train_size = int(0.7 * len(ratings_sorted))

train_df = ratings_sorted.iloc[:train_size].copy()
test_df = ratings_sorted.iloc[train_size:].copy()

print(f'Размер обучающей выборки: {len(train_df)}')
print(f'Размер тестовой выборки: {len(test_df)}')

Размер обучающей выборки: 70585
Размер тестовой выборки: 30251


In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [76]:
all_users = sorted(ratings['userId'].unique())
all_items = sorted(ratings['movieId'].unique())
user_to_index = {user: i for i, user in enumerate(all_users)}
item_to_index = {item: i for i, item in enumerate(all_items)}

In [26]:
n_users = len(all_users)
n_items = len(all_items)

R_train = np.zeros((n_users, n_items))
for _, row in train_df.iterrows():
    u = user_to_index[row['userId']]
    i = item_to_index[row['movieId']]
    R_train[u, i] = row['rating']

R_train.shape

(610, 9724)

# 2. Реализация SVD через SGD

In [34]:
K = 50  # Количество латентых факторов
learning_rate = 0.005
regularization_param = 0.02
epochs = 20

In [36]:
P = np.random.rand(n_users, K) * 0.1
Q = np.random.rand(n_items, K) * 0.1

mu = np.mean(R_train[R_train > 0])

b_u = np.zeros(n_users) # баес пользователя
b_i = np.zeros(n_items) # баес предмета

print('Начало обучения Funk SVD')

for epoch in range(epochs):
    for u_idx in range(n_users):
        for i_idx in range(n_items):
            if R_train[u_idx, i_idx] > 0:
                r_ui = R_train[u_idx, i_idx]
                # Предсказание рейтинга по формуле: r = mu + b_u + b_i + P_u * Q_i^T
                r_hat_ui = mu + b_u[u_idx] + b_i[i_idx] + np.dot(P[u_idx, :], Q[i_idx, :])

                error = r_ui - r_hat_ui
                
                # Обновление баесов
                b_u[u_idx] += learning_rate * (error - regularization_param * b_u[u_idx])
                b_i[i_idx] += learning_rate * (error - regularization_param * b_i[i_idx])
                
                # Обновление матриц латентных факторов
                P[u_idx, :] += learning_rate * (error * Q[i_idx, :] - regularization_param * P[u_idx, :])
                Q[i_idx, :] += learning_rate * (error * P[u_idx, :] - regularization_param * Q[i_idx, :])

print('Обучение Funk SVD завершено')

Начало обучения Funk SVD...
Обучение Funk SVD завершено.


In [38]:
R_pred = mu + b_u[:, np.newaxis] + b_i[np.newaxis, :] + np.dot(P, Q.T)

# 3. Функции расчета метрик

In [50]:
def calculate_average_precision_at_k(recommended_items, relevant_items, k):
    '''Рассчитывает Average Precision@K (AP@K).'''
    if not relevant_items:
        return 0.0
    
    score = 0.0
    num_hits = 0.0
    
    for i in range(k):
        item = recommended_items[i]
        if item in relevant_items:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(relevant_items), k)

In [52]:
def calculate_dcg_at_k(recommended_items, relevant_items_scores, k):
    '''Рассчитывает DCG@K. relevant_items_scores - словарь {item: score}.'''
    dcg = 0.0
    for i in range(k):
        item = recommended_items[i]
        if item in relevant_items_scores:
            gain = 2**relevant_items_scores[item] - 1
        else:
            gain = 0
        dcg += gain / np.log2(i + 2) # индексация с 0 поэтому 2 а не 1
    return dcg

In [54]:
def calculate_ndcg_at_k(recommended_items, relevant_items_scores, k):
    '''Рассчитывает NDCG@K.'''
    dcg = calculate_dcg_at_k(recommended_items, relevant_items_scores, k)
    idcg_gains = sorted(
        [2**score - 1 for score in relevant_items_scores.values()], 
        reverse=True)[:k]
    
    idcg = 0.0
    for i, gain in enumerate(idcg_gains):
        idcg += gain / np.log2(i + 2)
        
    # 3. NDCG
    if idcg == 0:
        return 0.0
    return dcg / idcg

# Расчет метрик на тестовой выборке

In [70]:
K_METRIC = 50
RELEVANCE_THRESHOLD = 4

In [72]:
test_user_ratings = test_df.groupby('userId').apply(
    lambda x: dict(zip(x['movieId'], x['rating']))
).to_dict()

test_users = [u for u in test_user_ratings.keys() if u in user_to_index] # берем пользоваетелей которые есть в трейне

C:\Users\User\AppData\Local\Temp\ipykernel_19396\1513054882.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_user_ratings = test_df.groupby('userId').apply(


In [74]:
all_ap_scores = []
all_ndcg_scores = []

print(f'Расчет MAP@{K_METRIC} и NDCG@{K_METRIC} для SVD...')

for user_id in test_users:
    u_idx = user_to_index[user_id]
    rated_items_train_indices = np.where(R_train[u_idx, :] > 0)[0] # ищем уже оцененные пользователем предметы
    user_predictions = R_pred[u_idx, :] # предсказания для всех предметов i пользователя

    temp_predictions = user_predictions.copy()
    temp_predictions[rated_items_train_indices] = -np.inf # искл уже оцененные пользователем предметы

    recommended_indices = np.argsort(temp_predictions)[::-1] # индексы лучших предсказаний
    index_to_item = {i: item for item, i in item_to_index.items()}
    recommended_items_id = [index_to_item[i] for i in recommended_indices]
    
    relevant_items_test_scores = test_user_ratings[user_id]

    relevant_items_id = {
        item: score 
        for item, score in relevant_items_test_scores.items() 
        if score >= RELEVANCE_THRESHOLD
    }

    ap_score = calculate_average_precision_at_k(
        recommended_items_id, 
        list(relevant_items_id.keys()), 
        K_METRIC
    )
    all_ap_scores.append(ap_score)

    ndcg_score = calculate_ndcg_at_k(
        recommended_items_id, 
        relevant_items_id,
        K_METRIC
    )
    all_ndcg_scores.append(ndcg_score)

MAP_K_SVD = np.mean(all_ap_scores)
NDCG_K_SVD = np.mean(all_ndcg_scores)

print(f'MAP@{K_METRIC}: {MAP_K_SVD:.4f}')
print(f'NDCG@{K_METRIC}: {NDCG_K_SVD:.4f}')

Расчет MAP@50 и NDCG@50 для SVD...

--- Результаты SVD (Funk SVD) ---
MAP@50: 0.0851
NDCG@50: 0.2019


# Сравнение с item-based и user-based из первой лабы

In [110]:
# функция косинусной близости
def cosine_similarity(vec1, vec2): 
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

Ниже код из первой лабы для сравнения метрик

## User-based, расчет новых метрик

In [112]:
train_data, test_data = train_test_split(ratings, test_size=0.3, random_state=42)
unique_users = np.unique(ratings.to_numpy()[:, 0]) # уникальные юзеры
unique_movies = np.unique(ratings.to_numpy()[:, 1]) # уникальные фильмы
user_id_dict = {user_id: i for i, user_id in enumerate(unique_users)}
movie_id_dict = {movie_id: i for i, movie_id in enumerate(unique_movies)}

In [116]:
#user-based из первой лабы
user_item_matrix = np.zeros((len(unique_users), len(unique_movies)))
for user_id, movie_id, rating, _ in train_data.to_numpy():
    user_idx = user_id_dict[user_id]
    movie_idx = movie_id_dict[movie_id]
    user_item_matrix[user_idx, movie_idx] = rating

# расчет матрицы сходства
n_users = len(unique_users)
user_similarity_matrix = np.zeros((n_users, n_users))

print('Начало расчета полной матрицы сходства пользователей')
for u_idx in range(n_users):
    for v_idx in range(u_idx, n_users): 
        if u_idx == v_idx:
            user_similarity_matrix[u_idx, v_idx] = 1.0
        else:
            sim = cosine_similarity(user_item_matrix[u_idx, :], user_item_matrix[v_idx, :]) 
            user_similarity_matrix[u_idx, v_idx] = sim
            user_similarity_matrix[v_idx, u_idx] = sim # Заполняем симметрично

print('Матрица сходства пользователей готова')


def predict_user_based_optimized(user_id, movie_id):
    if user_id not in user_id_dict or movie_id not in movie_id_dict:
        return np.nan
    
    u_idx = user_id_dict[user_id]
    i_idx = movie_id_dict[movie_id]
    u_ratings = user_item_matrix[u_idx, :]
    r_u_bar = u_ratings[u_ratings > 0].mean() if np.any(u_ratings > 0) else 0.0
    v_indices_rated_item = np.where(user_item_matrix[:, i_idx] > 0)[0]
    v_indices_rated_item = v_indices_rated_item[v_indices_rated_item != u_idx] # Исключаем u
    
    if len(v_indices_rated_item) == 0:
        return r_u_bar
    all_similarities = user_similarity_matrix[u_idx, v_indices_rated_item]
    sorted_indices = np.argsort(np.abs(all_similarities))[::-1]
    top_n_indices = v_indices_rated_item[sorted_indices[:N_NEIGHBORS]]
    
    chisl = 0.0
    znam = 0.0
    
    for v_idx in top_n_indices:
        sim_uv = user_similarity_matrix[u_idx, v_idx]

        r_v_bar = user_item_matrix[v_idx, :][user_item_matrix[v_idx, :] > 0].mean() 
        r_vi = user_item_matrix[v_idx, i_idx]
        
        chisl += sim_uv * (r_vi - r_v_bar)
        znam += abs(sim_uv)
        
    if znam == 0:
        return r_u_bar
    predicted_rating = r_u_bar + (chisl / znam)
    return np.clip(predicted_rating, 1.0, 5.0)

Начало расчета полной матрицы сходства пользователей
Матрица сходства пользователей готова


In [120]:
N_NEIGHBORS = 50

test_user_ratings = test_data.groupby(test_data.columns[0]).apply(
    lambda x: dict(zip(x.iloc[:, 1], x.iloc[:, 2]))
).to_dict()

test_users_cf = [u for u in test_user_ratings.keys() if u in user_id_dict]
all_items_list = list(movie_id_dict.keys())

all_ap_scores_cf = []
all_ndcg_scores_cf = []

print(f'Начало расчета MAP@{K_METRIC} и NDCG@{K_METRIC} для User-Based CF')

for user_id in test_users_cf:
    user_predictions_map = {}
    user_idx = user_id_dict[user_id]
    rated_items_indices_train = np.where(user_item_matrix[user_idx, :] > 0)[0] # Исключение фильмов, оцененных в TRAIN
    index_to_movie_id = {idx: movie_id for movie_id, idx in movie_id_dict.items()}
    rated_items_id_train = {index_to_movie_id[idx] for idx in rated_items_indices_train}

    for movie_id in all_items_list:
        if movie_id not in rated_items_id_train:
            predicted_rating = predict_user_based_optimized(user_id, movie_id) 
            if not np.isnan(predicted_rating):
                user_predictions_map[movie_id] = predicted_rating

    # Ранжирование
    recommended_items_id_cf = sorted(
        user_predictions_map, 
        key=user_predictions_map.get, 
        reverse=True
    )[:K_METRIC]

    relevant_items_test_scores = test_user_ratings[user_id]
    
    relevant_items_id_list = [item for item, score in relevant_items_test_scores.items() if score >= RELEVANCE_THRESHOLD]
    relevant_items_id_scores = {item: score for item, score in relevant_items_test_scores.items() if score >= RELEVANCE_THRESHOLD}

    all_ap_scores_cf.append(calculate_average_precision_at_k(recommended_items_id_cf, relevant_items_id_list, K_METRIC))
    all_ndcg_scores_cf.append(calculate_ndcg_at_k(recommended_items_id_cf, relevant_items_id_scores, K_METRIC))

# ФИНАЛЬНЫЕ МЕТРИКИ
MAP_K_CF_USER = np.mean(all_ap_scores_cf)
NDCG_K_CF_USER = np.mean(all_ndcg_scores_cf)

print('Результаты User-Based CF')
print(f'MAP@{K_METRIC}: {MAP_K_CF_USER:.4f}')
print(f'NDCG@{K_METRIC}: {NDCG_K_CF_USER:.4f}')

C:\Users\User\AppData\Local\Temp\ipykernel_19396\4120490060.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_user_ratings = test_data.groupby(test_data.columns[0]).apply(


Начало расчета MAP@50 и NDCG@50 для User-Based CF

--- Результаты User-Based CF (Оптимизированный) ---
MAP@50: 0.0008
NDCG@50: 0.0049


## item-based, расчет новых метрик

In [124]:
item_user_matrix = np.zeros((len(unique_movies), len(unique_users)))
for user_id, movie_id, rating, _ in train_data.to_numpy():
    user_idx = user_id_dict[user_id]
    movie_idx = movie_id_dict[movie_id]
    item_user_matrix[movie_idx, user_idx] = rating

n_items = len(unique_movies)
item_similarity_matrix = np.zeros((n_items, n_items))

print('Начало расчета полной матрицы сходства предметов')

for i_idx in range(n_items):
    for j_idx in range(i_idx, n_items):
        if i_idx == j_idx:
            item_similarity_matrix[i_idx, j_idx] = 1.0
        else:
            # сравниваем векторы оценок фильмов)
            sim = cosine_similarity(item_user_matrix[i_idx, :], item_user_matrix[j_idx, :]) 
            item_similarity_matrix[i_idx, j_idx] = sim
            item_similarity_matrix[j_idx, i_idx] = sim # Заполняем симметрично

print('Матрица сходства предметов готова')
print(f'Размер матрицы сходства: {item_similarity_matrix.shape}')

Начало расчета полной матрицы сходства предметов
Матрица сходства предметов готова
Размер матрицы сходства: (9724, 9724)


In [128]:
N_NEIGHBORS = 50 

def predict_item_based_optimized(user_id, movie_id):
    if user_id not in user_id_dict or movie_id not in movie_id_dict:
        return np.nan
    
    u_idx = user_id_dict[user_id]
    i_idx = movie_id_dict[movie_id]
    i_ratings = item_user_matrix[i_idx, :]
    r_i_bar = i_ratings[i_ratings > 0].mean() if np.any(i_ratings > 0) else 0.0

    j_indices_rated_by_user = np.where(item_user_matrix[:, u_idx] > 0)[0]
    j_indices_rated_by_user = j_indices_rated_by_user[j_indices_rated_by_user != i_idx]
    
    if len(j_indices_rated_by_user) == 0:
        return r_i_bar
    all_similarities = item_similarity_matrix[i_idx, j_indices_rated_by_user]
    sorted_indices = np.argsort(np.abs(all_similarities))[::-1]
    top_n_indices = j_indices_rated_by_user[sorted_indices[:N_NEIGHBORS]]
    
    chisl = 0.0
    znam = 0.0
    
    for j_idx in top_n_indices:
        sim_ij = item_similarity_matrix[i_idx, j_idx]
        r_j_bar = item_user_matrix[j_idx, :][item_user_matrix[j_idx, :] > 0].mean() 
        r_ju = item_user_matrix[j_idx, u_idx]
        
        chisl += sim_ij * (r_ju - r_j_bar)
        znam += abs(sim_ij)
        
    if znam == 0:
        return r_i_bar
    
    predicted_rating = r_i_bar + (chisl / znam)
    return np.clip(predicted_rating, 1.0, 5.0)

In [130]:
all_ap_scores_cf_item = []
all_ndcg_scores_cf_item = []

print(f"Начало расчета MAP@{K_METRIC} и NDCG@{K_METRIC}.")

for user_id in test_users_cf:
    user_predictions_map = {}
    u_idx = user_id_dict[user_id]

    index_to_movie_id = {idx: movie_id for movie_id, idx in movie_id_dict.items()}
    rated_items_indices_train = np.where(item_user_matrix[:, u_idx] > 0)[0] 
    rated_items_id_train = {index_to_movie_id[idx] for idx in rated_items_indices_train}

    for movie_id in all_items_list:
        if movie_id not in rated_items_id_train:
            predicted_rating = predict_item_based_optimized(user_id, movie_id) 
            if not np.isnan(predicted_rating):
                user_predictions_map[movie_id] = predicted_rating

    recommended_items_id_cf = sorted(
        user_predictions_map, 
        key=user_predictions_map.get, 
        reverse=True
    )[:K_METRIC]

    relevant_items_test_scores = test_user_ratings[user_id]
    relevant_items_id_list = [item for item, score in relevant_items_test_scores.items() if score >= RELEVANCE_THRESHOLD]
    relevant_items_id_scores = {item: score for item, score in relevant_items_test_scores.items() if score >= RELEVANCE_THRESHOLD}

    all_ap_scores_cf_item.append(calculate_average_precision_at_k(recommended_items_id_cf, relevant_items_id_list, K_METRIC))
    all_ndcg_scores_cf_item.append(calculate_ndcg_at_k(recommended_items_id_cf, relevant_items_id_scores, K_METRIC))

MAP_K_CF_ITEM = np.mean(all_ap_scores_cf_item)
NDCG_K_CF_ITEM = np.mean(all_ndcg_scores_cf_item)

print("\n Результаты")
print(f"MAP@{K_METRIC}: {MAP_K_CF_ITEM:.4f}")
print(f"NDCG@{K_METRIC}: {NDCG_K_CF_ITEM:.4f}")

Начало расчета MAP@50 и NDCG@50.

 Результаты
MAP@50: 0.0005
NDCG@50: 0.0031


# 4. Сравнение подходов

Сравнение показало, что Матричная Факторизация (SVD) **значительно превосходит** подходы коллаборативной фильтрации, основанные на близости (User-based и Item-based), по всем метрикам качества.

Ключевое преимущество SVD заключается в эффективном **преодолении разреженности данных**: она использует латентные факторы для обобщения предпочтений, что позволяет делать более точные прогнозы, чем простые корреляции между явными оценками. 
User/Item-based методы критически зависят от наличия явных, пересекающихся оценок, SVD более масштабируема и надежна.

Но SVD имеет существенный недостаток: она страдает от проблемы "холодного старта" для новых сущностей и предлагает рекомендации с низкой интерпретируемостью.

Таким образом, SVD является наиболее точным базовым алгоритмом, но требует гибридизации для полного решения проблем реального мира.